[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/028_pytorch_nn/pytorch_nn.ipynb)

# Pytorch - Redes Neuronales

En el post [anterior](https://sensioai.com/blog/027_pytorch_intro) hicimos una introducción al framework de `redes neuronales` `Pytorch`. Hablamos de sus tres elementos fundamentales: el objeto `tensor` (similar al `array` de `NumPy`) `autograd` (que nos permite calcular derivadas de manera automáticas) y el soporte GPU. En este post vamos a entrar en detalle en la  funcionalidad que nos ofrece la librería para diseñar redes neuronales de manera flexible.

In [40]:
import torch

## Modelos secuenciales

La forma más sencilla de definir una `red neuronal` en `Pytorch` es utilizando la clase `Sequentail`. Esta clase nos permite definir una secuencia de capas, que se aplicarán de manera secuencial (las salidas de una capa serán la entrada de la siguiente). Ésto ya lo conocemos de posts anteriores, ya que es la forma ideal de definir un `Perceptrón Multicapa`.

In [41]:
D_in, H1, H2, D_out = 784, 100, 50, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.ReLU(),
    torch.nn.Linear(H1, H2),
    torch.nn.ReLU(),
    torch.nn.Linear(H2, D_out),
)

El modelo anterior es un `MLP` con 784 entradas, 100 neuronas en la capa oculta y 10 salidas. Podemos usar este modelo para hacer un clasificador de imágenes con el dataset MNIST. Pero primero, vamos a ver como podemos calcular las salidas del modelo a partir de unas entradas de ejemplo.

In [42]:
outputs = model(torch.randn(64, 784))
outputs.shape

torch.Size([64, 10])

Como puedes ver, simplemente le pasamos los inputs al modelo (llamándolo como una función). En este caso, usamos un tensor con 64 vectores de 784 valores. Es importante remarcar que los modelos de `Pytorch` (por lo general) siempre esperan que la primera dimensión sea la dimensión *batch*. Si queremos entrenar esta red en una GPU, es tan sencillo como

In [43]:
model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=10, bias=True)
)

Vamos a ver ahora como entrenar este modelo con el dataset MNIST.

In [44]:
import pandas as pd

data = pd.read_csv("/content/emnist.csv")

print(data.shape)
y = data[data.columns[0]]
x = data[data.columns[1:785]]
y = np.array(y)
x = np.array(x)
print(y.shape)
print(x.shape)

(18799, 785)
(18799,)
(18799, 784)


In [35]:
import numpy as np

# normalización y split

X_train, X_test, y_train, y_test = X[:15000] / 255., X[15000:] / 255., Y[:15000].astype(np.int), Y[15000:].astype(np.int)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test) 

<ipython-input-35-4a166dc07f2d>:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train, X_test, y_train, y_test = X[:15000] / 255., X[15000:] / 255., Y[:15000].astype(np.int), Y[15000:].astype(np.int)


In [36]:
# función de pérdida y derivada

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [45]:
# convertimos datos a tensores y copiamos en gpu
X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

# bucle entrenamiento
epochs = 500
lr = 0.9
log_each = 10
l = []
for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = cross_entropy(y_pred, Y_t)
    l.append(loss.item())
    
    # ponemos a cero los gradientes
    model.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 10/500 Loss 2.15910
Epoch 20/500 Loss 2.00951
Epoch 30/500 Loss 1.86740
Epoch 40/500 Loss 1.65752
Epoch 50/500 Loss 1.52898
Epoch 60/500 Loss 1.39166
Epoch 70/500 Loss 1.26216
Epoch 80/500 Loss 1.15690
Epoch 90/500 Loss 1.06202
Epoch 100/500 Loss 0.99041
Epoch 110/500 Loss 0.92373
Epoch 120/500 Loss 0.86407
Epoch 130/500 Loss 0.81342
Epoch 140/500 Loss 0.78230
Epoch 150/500 Loss 0.74795
Epoch 160/500 Loss 0.71160
Epoch 170/500 Loss 0.67866
Epoch 180/500 Loss 0.64884
Epoch 190/500 Loss 0.62465
Epoch 200/500 Loss 0.60505
Epoch 210/500 Loss 0.58210
Epoch 220/500 Loss 0.56082
Epoch 230/500 Loss 0.54109
Epoch 240/500 Loss 0.52275
Epoch 250/500 Loss 0.50569
Epoch 260/500 Loss 0.48996
Epoch 270/500 Loss 0.50354
Epoch 280/500 Loss 0.51931
Epoch 290/500 Loss 0.52334
Epoch 300/500 Loss 0.52251
Epoch 310/500 Loss 0.51184
Epoch 320/500 Loss 0.50077
Epoch 330/500 Loss 0.48983
Epoch 340/500 Loss 0.49701
Epoch 350/500 Loss 0.50194
Epoch 360/500 Loss 0.50233
Epoch 370/500 Loss 0.49542
Epoch 380/

Como puedes observar en el ejemplo, podemos calcular la salida del modelo con una simple línea. Luego calculamos la función de pérdida, y llamando a la función `backward` `Pytorch` se encarga de calcular las derivadas de la misma con respecto a todos los parámetros del modelo automáticamente (si no queremos acumular estos gradientes, nos aseguramos de llamar a la función `zero_grad` para ponerlos a cero antes de calcularlos). Por útlimo, podemos iterar por los parámetros del modelo aplicando la regla de actualización deseada (en este caso usamos `descenso por gradiente`).

In [46]:
from sklearn.metrics import accuracy_score

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

0.9483272727272727